<!--

Analysis of the results with graphics like bargraphs and boxplots has been made. Statistical methods have been employed to compare different results. AUCs have been obtained. Minimal verification.
+ Attention has been paid to the distribution of the data. Verification has been carried out. Attention is paid to overfitting.
+ Results have been analysed with proper statistics. Model assumptions of statistical methods are verified or methods that do not make assumptions were employed for non-Gaussian data. Checks were made to avoid hidden overfitting.
+ Best approach for obtaining significant results was taken while refraining from p-value hacking.

 -->

## Results

### Evaluation of methods

#### Regression {#sec-res-reg}

| Model          | Hyp. opt. obj. | RMSE          | Bias         |
| -------------- | -------------- | ------------- | ------------ |
| OLS            | min RMSE       | 110.6 ± 24.2  | -18.8 ± 10.4 |
| OLS            | min bias       | 130.9 ± 21.5  | -14.3 ± 10.1 |
| BayesianRidge  | min bias       | 119.7 ± 23.4  | -3.85 ± 15.2 |
| LassoLarsIC    | min bias       | 118.9 ± 24.5  | -2.89 ± 15.7 |

: Using 5-fold time-series split. (Results for using conventional cross-validation split are approximately unbiased (bias < 0.02).) {#tbl-reg-hypopt}

Hyperparameter optimization shows that the best OLS regression model uses 7 time-series lags of all variables and no additional features. On 5 time-series cross-validation splits, it achieves a predictive performance with a root mean squared error (RMSE) of 36.5±6.2 and a (cross-validated) R² of 0.88±0.02. (The dimension is the sum of the number of climate articles published within a given date and the following day.) This is almost identical to a similarly tuned Bayesian Ridge regression model, which achieves an RMSE of 36.5±6.3 and the same R² value. A detailed regression table for the best OLS model can be found in @sec-app1-reg.


In [ ]:
# | echo: False

from src.models.regression.hyperopt import best_regression

r = best_regression(ignore_group=True)
t = "occ_protest_lag0"
results = r.params[t], tuple(r.conf_int().loc[t]), r.pvalues[t]
results

The coefficients for the protest variables are equivalent to the ATT (see @sec-app2-reg). Statistically significant at p=0.05 are only protest events by Fridays for Future, Letzte Generation, and groups that are coded as "other" groups. All protest coefficients are positive except for Ende Gelände and Fridays for Future + X, with negative coefficients at p-values of 0.89 and 0.18 respectively. For the aggregated protest occurrence variable (which does not consider the organizing protest group) the estimate is +5.79 additional articles in the first two days, with a confidence interval [+2.67, +8.92] and a p-value of 0.0003. More estimates are displayed in @sec-res-est.

<!-- MSE generalizability -->

#### Instrumental variables

Out of the 15 potential instrumental variables, 5 pandemic instruments and 4 weather instruments have a statistically significant impact on protest occurrence (see @fig-iv-basic), but when using them in a combined regression the coefficients are much less significant. Precipitation is not among the significant variables.

Automatically binarizing the variables based on an optimally chosen threshold does not generally increase the coefficients and siginificances, and decreases them slightly in most cases. This is also true for precipiation: The optimal threshold for maximizing the covariance with protest occurrence is at prcp > 0, which is consistent with the choice in most related work (see @sec-weatherlit). I find that such binarization is not systematically better or worse than using the continuous value. (For the single regression, it slightly decreases the coefficient, and for the combined regression with other variables it slightly increases the coefficient.)


In [ ]:
# | echo: false
# | fig-cap: Principal components of the
from src.models.instrumental_variable import pc_vis

pc_vis("pc_weather_covid", numbers=[0, 7, 11])

<!-- pc_0 at p=0.0005, pc_7 at p=0.007, and pc_11 at p=0.008 -->

Principal component decomposition isolates three very significant components with p<0.01, while the other components have p>0.05.^[This is without adjusting for multiple testing since no hypothesis tests or thresholds are used, but the analysis is rather exploratory.] The three components are displayed in terms of the original variables in @fig-iv-pc.


In [ ]:
# | echo: false
# | fig-cap: Principal components of the
from src.models.instrumental_variable import pc_vis

pc_vis("pc_weather_covid_season", numbers=[9, 8, 11], dfs=slice(1, 2))

Deseasoning the original variables and then performing separate principle component analyses on seasonal and residual parts shows that most of the significant variables are seasonal in nature; but there is also one very significant residual variable pc_resid_9 at p=0.0005 and two somewhat significant residual variables pc_resid_8 at p=0.05 and pc_resid_11 at p=0.07, while all other residual components have p>0.15 (without adjusting for multiple testing). The more significant components are broken down in @fig-iv-pc-deseasoned. The first-stage f-statistic for pc_resid_9 is f=33.82; f=2.08 for pc_resid_8; and f=2.02. According to the "rule of thumb" (@InstrumentalVariablesEstimation2023) the threshold for weak instruments is around f=10, so pc_resid_9 would be a strong instrument but the other ones would all be weak. The combined f-statistic of the three mentioned components is f=38.74.

Placebo tests for the first stage (the impact of the instrument on the treatment) are given in @fig-iv-1-placebo. From day 3 after the protest date there is no longer a significant impact of the instrument on the treatment


In [ ]:
import matplotlib.pyplot as plt

# | fig-cap: First stage placebo tests for using the principal component pc_resid_9 to instrument protest occurrence. Values for the instrument from after the protest date are used. These values cannot have an impact on protest occurrence if the instrument is valid.
# | echo: false
from src.models.time_series import instrumental_variable_liml

results = instrumental_variable_liml(
    target="media_combined_protest",
    treatment="occ_protest",
    instruments="pc_weather_covid_season",
    iv_instruments=["pc_resid_9"],
    lags=range(-7, 1),
    ignore_group=True,
    ignore_medium=True,
    cumulative=False,
    steps=range(-7, 29),
    shift_instruments=True,
    show_progress=False,
)
fig, ax = plt.subplots(figsize=(6, 3))
ax.plot(results.step, results.first_stage_coef, marker=".")
ax.fill_between(
    results.step,
    results.first_stage_ci_lower,
    results.first_stage_ci_upper,
    alpha=0.2,
    color="C0",
)
ax.axhline(0, color="k", linewidth=0.5)
ax.axvline(0, color="k", linestyle="--")
ax.set_xlabel("Day of instrument (relative to protest)")
ax.set_ylabel("First stage coefficient")
ax.set_title(
    "First stage placebo tests with shifted outcome: Impact of pc_resid_9 on protest occurrence"
)
plt.tight_layout()
plt.show()

#### Synthetic control

180 lags
rmse 65.8+-2.40
bias
-0.330+-1.13


In [ ]:
# | echo: false

import matplotlib.pyplot as plt

from src.models.synthetic_control.hyperopt import hyperopt

results, _ = hyperopt(show_progress=False)
fig, ax = plt.subplots(figsize=(5, 2))
ax.plot([abs(r["lags"][0]) for r in results], [r["bias"] for r in results], marker=".")
plt.fill_between(
    [abs(r["lags"][0]) for r in results],
    [r["bias"] - r["bias_std"] for r in results],
    [r["bias"] + r["bias_std"] for r in results],
    alpha=0.2,
)
ax.axhline(0, color="black", linewidth=0.5)
ax.set_xlabel("Pre-period length")
ax.set_ylabel("Bias")
ax.set_title("Bias vs. pre-period length")
ax.set_xticks([abs(r["lags"][0]) for r in results])
plt.tight_layout()
plt.show()

In [ ]:
# | echo: false
import matplotlib.pyplot as plt

from src.models.synthetic_control import sc_plot

titlenames = [
    ("articles mentioning protest", "media_combined_protest"),
    ("articles NOT mentioning protest", "media_combined_not_protest"),
]
for t, n in titlenames:
    fig, ax = sc_plot(t, n, scale="demean", pre_period=22, post_period=22)
    ax.set_xlim(-25, 25)
    plt.show()

In [ ]:
# | echo: false
# | fig-cap: with 7-day rolling average

for t, n in titlenames:
    fig, ax = sc_plot(
        t, n, scale="demean", rolling=7, pre_period=365, post_period=3 * 28
    )
    plt.show()

Difficulties with large protest events: plot size vs #regions

#### Propensity scores

hypopt n lags, ewm, diffs, log sizes

predicting all positive: 0.118

logistic regression unbalanced with diffs, ewms, sizes, 4 lags

F1: 0.233±0.044
balanced: 0.213+-0.017

### Placebo tests

#### Placebo outcome


In [ ]:
# | column: page
# | echo: false
import matplotlib.pyplot as plt

from src.visualization.impacts import plot_trends

fig, axes = plot_trends(cumulative=False, steps=range(-28, 8), show_progress=False)
fig.suptitle("Placebo test: estimating the causal impact on outcomes before the event")
axes[0].set_xticks(range(-28, 8, 7))
axes[0].set_ylim(-30, 30)
axes[1].set_ylim(-30, 30)
axes[2].set_ylim(-5, 5)
axes[3].set_ylim(-5, 5)
axes[4].set_ylim(-300, 300)
plt.tight_layout()
plt.show()

#### Placebo treatment


In [ ]:
# | column: page
# | echo: false
import matplotlib.pyplot as plt

from src.visualization.impacts import plot_trends

fig, axes = plot_trends(
    cumulative=False,
    steps=range(-15, 15),
    random_treatment_global=56,
    show_progress=False,
)
fig.suptitle("Placebo test: estimating the causal impact of random events")
# axes[0].set_ylim(-3, 5)
axes[0].set_xticks(range(-14, 15, 7))
axes[0].set_ylim(-5, 5)
axes[1].set_ylim(-5, 5)
axes[2].set_ylim(-5, 5)
axes[3].set_ylim(-5, 5)
axes[4].set_ylim(-1000, 1000)
plt.tight_layout()
plt.show()

In [ ]:
# | column: page
# | echo: false
import matplotlib.pyplot as plt

from src.visualization.impacts import plot_trends

fig, axes = plot_trends(
    cumulative=False,
    steps=range(-15, 15),
    random_treatment_regional=25,
    show_progress=False,
)
fig.suptitle("Placebo test: estimating the causal impact of random events")
# axes[0].set_ylim(-3, 5)
axes[0].set_xticks(range(-14, 14, 7))
axes[0].set_ylim(-30, 30)
axes[1].set_ylim(-30, 30)
axes[2].set_ylim(-5, 5)
axes[3].set_ylim(-5, 5)
axes[4].set_ylim(-3000, 3000)
plt.tight_layout()
plt.show()

In [ ]:
# | column: page
# | echo: false

# Does this even make so much sense, placebo for protest groups? maybe leave out.

# import altair as alt
# from src.visualization.impacts import plot_groups

# alt.renderers.enable("altair_saver", fmts=["html", "svg"])
# plot_groups(kind="methods", step=1, random_treatment_global=True)
# axes[0].set_ylim(-5, 5)
# axes[1].set_ylim(-5, 5)
# axes[2].set_ylim(-5, 5)
# axes[3].set_ylim(-5, 5)
# axes[4].set_ylim(-5, 5)

### Causal impact estimates {#sec-res-est}

#### Time series


In [ ]:
# | fig-cap: Note that the y-axes are shared between some of the plots, but the instrumental variable plot has a much larger y-axis.
# | column: page
# | echo: false
import matplotlib.pyplot as plt

from src.visualization.impacts import plot_trends

fig, axes = plot_trends(cumulative=False, show_progress=False)
fig.suptitle("Time series of the causal impact by method and coverage dimension")
axes[0].set_ylim(-30, 30)
axes[1].set_ylim(-30, 30)
axes[2].set_ylim(-5, 5)
axes[3].set_ylim(-5, 5)
axes[4].set_ylim(-300, 300)
plt.tight_layout()
plt.show()

#### Cumulative impact


In [ ]:
# | fig-cap: Note that the y-axes are shared between some of the plots, but the instrumental variable plot has a much larger y-axis.
# | column: page
# | echo: false
import matplotlib.pyplot as plt

from src.visualization.impacts import plot_trends

fig, axes = plot_trends(cumulative=True, show_progress=False)
fig.suptitle(
    "Cumulative time series of the causal impact by method and coverage dimension"
)
axes[0].set_ylim(-100, 300)
axes[1].set_ylim(-100, 300)
axes[2].set_ylim(-10, 30)
axes[3].set_ylim(-10, 30)
axes[4].set_ylim(-500, 1500)
plt.tight_layout()
plt.show()

#### Protest groups

Causal effects for the individual protest groups differ substantially between regression and inverse propensity weighting and are not statistically significant at p=0.05 for the doubly robust estimator.


In [ ]:
# | column: page
# | echo: false

import altair as alt

from src.visualization.impacts import plot_groups

alt.renderers.enable("altair_saver", fmts=["html", "svg"])
plot_groups(kind="groups", step=1)

In [ ]:
# | column: page
# | echo: false

import altair as alt

from src.visualization.impacts import plot_groups

alt.renderers.enable("altair_saver", fmts=["html", "svg"])
plot_groups(kind="groups", step=6)

In [ ]:
# | column: page
# | echo: false

# problem: y axis visualization; maybe switch to matplotlib
import altair as alt

from src.visualization.impacts import plot_groups

alt.renderers.enable("altair_saver", fmts=["html", "svg"])
plot_groups(kind="methods", step=6)

#### Newspaper sources

#### Protest event sources

TODO